based on https://www.kaggle.com/competitions/word2vec-nlp-tutorial/overview

In [39]:
!cp kaggle.json /content
!chmod 600 /content/kaggle.json

In [1]:
import numpy as np
import pandas as pd 

In [36]:
!pip install kaggle --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 kB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.13-py3-none-any.whl size=77717 sha256=3277eda9a26c5478a2d03adf2156264a881b51150cccc67ee3db33b3120e5db6
  Stored in directory: /root/.cache/pip/wheels/92/8c/37/96a1971bedc1e74057af1e4852f18de7e8286dea4f12928e6c
Successfully built kaggle


In [2]:

## set up custom path to kaggle.json: 
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

from pathlib import Path
path = Path('word2vec-nlp-tutorial')
if not path.exists():
    import zipfile,kaggle
    kaggle.api.competition_download_cli(str(path))
    # for each zip file in the path unzip it
    for f in path.glob('*.zip'):
        zipfile.ZipFile(f).extractall(path)

In [3]:
train =  pd.read_csv(path/'labeledTrainData.tsv',sep='\t')

In [4]:
train.describe(include='object')

,id,review
count,25000,25000
unique,25000,24904
top,5814_8,"When i got this movie free from my job, along ..."
freq,1,3


In [5]:
train.review[0]

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [6]:
from bs4 import BeautifulSoup

In [7]:
bs1 = BeautifulSoup(train.review[0])
bs1.get_text()

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 20 mi

In [8]:
import re 

letters_only = re.sub("[^a-zA-Z]", " ", bs1.get_text())
letters_only

'With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on for    mi

In [11]:
words = letters_only.lower().split()
words[:5],len(words)

(['with', 'all', 'this', 'stuff', 'going'], 437)

In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
from nltk.corpus import stopwords 

In [15]:
words = [w for w in words if not w in stopwords.words("english")]
words[:5],len(words)

(['stuff', 'going', 'moment', 'mj', 'started'], 219)

In [16]:
import re 
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords 

def clean_review(review):
    bs = BeautifulSoup(review)
    letters_only = re.sub("[^a-zA-Z]", " ", bs.get_text())
    words = letters_only.lower().split()
    stops = set(stopwords.words("english"))
    meaningful_words = [w for w in words if not w in stops]
    return( " ".join( meaningful_words ))

In [17]:
clean_review(train.review[0])

'stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate workin

In [18]:
train['clean_review'] = train.review.apply(clean_review)

/usr/local/lib/python3.9/dist-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [19]:
train['review'][0], train['clean_review'][0]

("With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally st

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, 
                             preprocessor = None, stop_words = None, max_features = 5000)

train_data_features = vectorizer.fit_transform(train.clean_review)
train_data_features


<25000x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 1975048 stored elements in Compressed Sparse Row format>

In [21]:
train_data_features = train_data_features.toarray()
train_data_features.shape

(25000, 5000)

In [23]:
vocab = vectorizer.get_feature_names()
vocab[:10]

['abandoned',
 'abc',
 'abilities',
 'ability',
 'able',
 'abraham',
 'absence',
 'absent',
 'absolute',
 'absolutely']

In [24]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100) 
forest = forest.fit( train_data_features, train["sentiment"] )
forest

RandomForestClassifier()

In [25]:
test = pd.read_csv(path/'testData.tsv',sep='\t')
test['clean_review'] = test.review.apply(clean_review)
test_data_features = vectorizer.transform(test.clean_review).toarray()
test_data_features.shape


/usr/local/lib/python3.9/dist-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


(25000, 5000)

In [29]:
test_data_features[3][:50]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

In [30]:
test.head()

,id,review,clean_review
0,12311_10,Naturally in a film who's main themes are of m...,naturally film main themes mortality nostalgia...
1,8348_2,This movie is a disaster within a disaster fil...,movie disaster within disaster film full great...
2,5828_4,"All in all, this is a movie for kids. We saw i...",movie kids saw tonight child loved one point k...
3,7186_2,Afraid of the Dark left me with the impression...,afraid dark left impression several different ...
4,12128_7,A very accurate depiction of small time mob li...,accurate depiction small time mob life filmed ...


In [31]:
test['sentiment'] = forest.predict(test_data_features)

In [42]:
test.head()

,id,review,clean_review,sentiment
0,12311_10,Naturally in a film who's main themes are of m...,naturally film main themes mortality nostalgia...,1
1,8348_2,This movie is a disaster within a disaster fil...,movie disaster within disaster film full great...,0
2,5828_4,"All in all, this is a movie for kids. We saw i...",movie kids saw tonight child loved one point k...,1
3,7186_2,Afraid of the Dark left me with the impression...,afraid dark left impression several different ...,1
4,12128_7,A very accurate depiction of small time mob li...,accurate depiction small time mob life filmed ...,1


In [33]:
test[['id','sentiment']].to_csv('submission.csv',index=False)

In [41]:
import kaggle
kaggle.api.competition_submit('submission.csv', 'Random Forest from sample notebook', 
                              'word2vec-nlp-tutorial')


100%|██████████| 227k/227k [00:00<00:00, 533kB/s]


Successfully submitted to Bag of Words Meets Bags of Popcorn

Score for the submission above is: 0.84508

lets continue with part 2 https://www.kaggle.com/competitions/word2vec-nlp-tutorial/overview/part-2-word-vectors